# Real-time Steps

In real-time mode, the FPGA lets Xylo run freely and only performs readout after receiving a timestep done interrupt.
This means sending input to Xylo from the FPGA is not supported in real-time mode.  Instead, the switches on the PCB must be configured such that Xylo receives input from the microphones on the PCB.


Open Dev Kit

* Call open_device() on a discovered device with Samna.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import rockpool
print(f'Rockpool version {rockpool.__version__}')
import samna
print(f'Samna version {samna.__version__}')

from rockpool.devices.xylo.syns65302 import config_from_specification, mapper, XyloMonitor

from rockpool.devices.xylo import find_xylo_hdks

from rockpool.transform.quantize_methods import channel_quantize
from rockpool.nn.modules import LIF, Linear
from rockpool.nn.combinators import Sequential

import numpy as np

# find_xylo_hdks() directly calls the Samna device finder and opens all available devices.
# When Samna opens a device, it will perform actions on the board. Such as, powering on the chip, resetting it,
# applying a default configuration, and configuring the FPGA for default operation.

xylo_hdks, xylo_support_modules, xylo_versions = find_xylo_hdks()
print(xylo_hdks, xylo_support_modules, xylo_versions)

Rockpool version 2.7.dev
Samna version 0.37.10.26+g11d2fc36e.dirty
from samna.xyloA3.configuration import InputInterfaceConfig
The connected Xylo HDK contains a Xylo A3. Importing `rockpool.devices.xylo.syns65302`
[<samna.xyloAudio3Boards.XyloAudio3TestBoard object at 0x74f5249a88b0>] [<module 'rockpool.devices.xylo.syns65302' from '/home/vleite/Software/rockpool/rockpool/devices/xylo/syns65302/__init__.py'>] ['syns65302']


## Build a model and map it to hardware

In [2]:
# simple model to test

net = Sequential(
    Linear((2, 7)),
    LIF((7, 7)),

    Linear((7, 4)),
    LIF(4),
)

print(net)

# net[0].weight *= 0.05
spec = mapper(net.as_graph())
Q_spec = spec
Q_spec.update(channel_quantize(**Q_spec))
config, is_valid, msg = config_from_specification(**Q_spec)

if not is_valid:
    print(msg)

ModSequential  with shape (2, 4) {
    Linear '0_Linear' with shape (2, 7)
    LIF '1_LIF' with shape (7, 7)
    Linear '2_Linear' with shape (7, 4)
    LIF '3_LIF' with shape (4, 4)
}


## Set up Samna graphs and configuration

* Create a buffer sink for events from the chip model source node. Readout events will be captured here.
* Set Xylo configuration as desired for PDM and network config
* Set Xylo TR_WRAP register for desired time step length with default 25 MHz clock
* Set Xylo to real-time mode
* Enable PCB switches for PDM microphones
* Set FPGA module to real-time mode
   * io.write_config(0x12, 0) and io.write_config(0x31, 2)

In [1]:
# create xylo monitor
dt = 0.01
xylo_node = xylo_hdks[0]
print(xylo_node)


# PDM
config.digital_frontend.mode = samna.xyloAudio3.DigitalFrontendMode.Pdm

# TR_WRAP register
config.time_resolution_wrap = int(0x7_9FF3)

# real-time mode
config.operation_mode = samna.xyloAudio3.OperationMode.RealTime

config.debug.debug_status_update_enable = 1
config.debug.debug_input_enable = 1
config.debug.monitor_signals_enable = 1

# PCB switches
# This is physically checked on the board

# inside XyloMonitor the buffers are created and FPGA modules are set
xylo_monitor = XyloMonitor(device=xylo_node, 
    config=config, 
    dt = dt,
    output_mode='Spike')

print(xylo_monitor)

print(xylo_monitor.config)

# Create a XyloSamna structure for evolving
modSamna = XyloSamna(node, config, dt = dt)

NameError: name 'xylo_hdks' is not defined

## Start processing and collect events

* Send TriggerProcessing event from Samna with the desired time step
* Collect readout events in the BufferSink attached to the model source node

In [2]:
# The notebook calls XyloMonitor.evolve() with some input data, which the evolve function will try to send. 
# This is not allowed in real-time mode.

import numpy as np
N = 1000

# in rd there is a AFE related record ('spike_in' or similar).
# It should be empty in silence and non empty when we make noise
out, _, rd = modSamna.evolve(input = np.zeros(a,b, N), record = True)


# out, _, rec = xylo_monitor.evolve(input_data=np.ones((N,2)))
print(out)
print(rd)


NameError: name 'modSamna' is not defined